In [15]:
import nltk
import numpy as np
import pandas as pd
import matplotlib as plt

In [53]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [54]:
df_train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [55]:
len(df_train)

7613

In [56]:
df_train.drop(columns=['location'], inplace = True)
df_train.head()

,id,keyword,text,target
0,1,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [57]:
df_train['keyword'].fillna(" ", inplace = True)
df_train.head()

,id,keyword,text,target
0,1,,Our Deeds are the Reason of this #earthquake M...,1
1,4,,Forest fire near La Ronge Sask. Canada,1
2,5,,All residents asked to 'shelter in place' are ...,1
3,6,,"13,000 people receive #wildfires evacuation or...",1
4,7,,Just got sent this photo from Ruby #Alaska as ...,1


In [58]:
from nltk.tokenize import TweetTokenizer

tok = TweetTokenizer()

def tokenize_tweets(df):
    final_tokenized = []
    for i in range(len(df)):
        tokenized = tok.tokenize(df['text'][i])
        final_tokenized.append(tokenized)
        
    return final_tokenized

df_train['tok tweets'] = tokenize_tweets(df_train)
df_train.head()

,id,keyword,text,target,tok tweets
0,1,,Our Deeds are the Reason of this #earthquake M...,1,"[Our, Deeds, are, the, Reason, of, this, #eart..."
1,4,,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, ., Canada]"
2,5,,All residents asked to 'shelter in place' are ...,1,"[All, residents, asked, to, ', shelter, in, pl..."
3,6,,"13,000 people receive #wildfires evacuation or...",1,"[13,000, people, receive, #wildfires, evacuati..."
4,7,,Just got sent this photo from Ruby #Alaska as ...,1,"[Just, got, sent, this, photo, from, Ruby, #Al..."


In [59]:
from nltk.corpus import stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [60]:
def remove_stop_words():
    for i in range(len(df_train)):
        original_line = df_train['tok tweets'].iloc[i]
        stop_words = set(stopwords.words('english'))
        final_line = [word for word in original_line if word.lower() not in stop_words]
        df_train['tok tweets'].iloc[i] = final_line
        # print(final_line)
remove_stop_words()

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [61]:
import re
def collect_hashtags():
    hashtags = []
    regex = "#(\w+)"
    
    for i in range(len(df_train)):
        original_line = df_train['text'].iloc[i]
        hashtag_list = re.findall(regex, original_line)
        # print(hashtag_list)
        for hashtag in hashtag_list:
            # print(hashtag)
            hashtags.append(hashtag)
        original_line_list = df_train['tok tweets'].iloc[i]
        for w in range(len(original_line_list)):
            if original_line_list[w][0] == "#":
                original_line_list[w] = original_line_list[w][1:]
        df_train['tok tweets'].iloc[i] =  original_line_list
        
    # print(original_line_list)
    return hashtags
        
hashtags = set(collect_hashtags())

In [62]:
hashtags

{'romance',
 'Miami',
 'PREPPERS',
 'FreeBitcoin',
 '6',
 'LDS',
 'in',
 'professional',
 'tcot',
 'StandForWolves',
 'deai',
 'Nursing',
 '3',
 'Extinction',
 'best',
 'Anarchy',
 'Bangalore',
 'breaking',
 'load',
 'amreading',
 'NYC',
 'PlannedParenthood',
 'AllTheNews',
 'videoclip',
 'energy',
 'Kuwait',
 'Snowden',
 'BADChoices',
 'LIKES',
 'IDFire',
 'JAGM',
 'Security',
 'collegeradi',
 'Workers',
 'lucky',
 'AR',
 'explodingkittens',
 'residualincome',
 'thisispublichealth',
 'mPERS',
 'nook',
 'europe',
 'RevItUp',
 'care',
 'NashsNewVideo',
 'AZ',
 'GGIndependencMessage',
 'nlwx',
 'drained',
 'justaguess',
 'NJ36',
 'sinkhole',
 'legionstrackandfield',
 'perspective',
 'nowplay',
 'topnews',
 'loveyouuuu',
 'Zionism',
 'MasterChef',
 'minions',
 'Models',
 'ineedexposure',
 'tomclancy',
 'global',
 'seeyouatamicos',
 'smallbusiness',
 'RWA',
 'anonymous',
 'trees',
 'ENGvAUS',
 'Truth',
 'Arsenal',
 'love',
 'WeLoveYouLouis',
 'badkitty',
 'CNBC',
 'muaytai',
 'date',
 'Nyt

In [63]:
def remove_mentions():
    regex = "@(\w+)"
    for i in range(len(df_train)):
        original_line = df_train['text'].iloc[i]
        original_line_list = df_train['tok tweets'].iloc[i]
        mention_list = re.findall(regex, original_line)
        df_train['tok tweets'].iloc[i] = [word for word in original_line_list if word[1:] not in mention_list]
        # print(df_train['tok tweets'].iloc[i])
        
remove_mentions()  

In [64]:
def remove_punctuations():
    # takes care of all the links as well as links are not alpha numeric
    for i in range(len(df_train)):
        original_line = df_train['tok tweets'].iloc[i]
        final_line = [word for word in original_line if word.isalnum()]
        df_train['tok tweets'].iloc[i] = final_line
        # print(final_line)
remove_punctuations()

In [65]:
df_train.head()

,id,keyword,text,target,tok tweets
0,1,,Our Deeds are the Reason of this #earthquake M...,1,"[Deeds, Reason, earthquake, May, ALLAH, Forgiv..."
1,4,,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, La, Ronge, Sask, Canada]"
2,5,,All residents asked to 'shelter in place' are ...,1,"[residents, asked, shelter, place, notified, o..."
3,6,,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, Ruby, Alaska, smoke, wildfi..."


In [66]:
def remove_shorts():
    for i in range(len(df_train)):
        original_line = df_train['tok tweets'].iloc[i]
        final_line = [word for word in original_line if len(word) > 3]
        df_train['tok tweets'].iloc[i] = final_line
        
remove_shorts()

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [67]:
df_train.head()

,id,keyword,text,target,tok tweets
0,1,,Our Deeds are the Reason of this #earthquake M...,1,"[Deeds, Reason, earthquake, ALLAH, Forgive]"
1,4,,Forest fire near La Ronge Sask. Canada,1,"[Forest, fire, near, Ronge, Sask, Canada]"
2,5,,All residents asked to 'shelter in place' are ...,1,"[residents, asked, shelter, place, notified, o..."
3,6,,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,,Just got sent this photo from Ruby #Alaska as ...,1,"[sent, photo, Ruby, Alaska, smoke, wildfires, ..."


In [69]:
from nltk.stem import WordNetLemmatizer

def lemmatize():
    lem = WordNetLemmatizer()
    for i in range(len(df_train)):
        original_line = df_train.loc[i, 'tok tweets']
        for j in range(len(original_line)):
            x = lem.lemmatize(original_line[j].lower())
            df_train.loc[i, 'tok tweets'][j] = x
        # print(original_line)
lemmatize()

In [70]:
df_train.head()

,id,keyword,text,target,tok tweets
0,1,,Our Deeds are the Reason of this #earthquake M...,1,"[deed, reason, earthquake, allah, forgive]"
1,4,,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, ronge, sask, canada]"
2,5,,All residents asked to 'shelter in place' are ...,1,"[resident, asked, shelter, place, notified, of..."
3,6,,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfire, evacuation, order,..."
4,7,,Just got sent this photo from Ruby #Alaska as ...,1,"[sent, photo, ruby, alaska, smoke, wildfire, p..."


In [23]:
df_test = pd.read_csv("test.csv")
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [24]:
df_test.isnull().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [25]:
len(df_test)

3263

In [26]:
df_test.drop(columns=['location'], inplace = True)
df_test.head()

,id,keyword,text
0,0,NaN,Just happened a terrible car crash
1,2,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
df_test['keyword'].fillna(" ", inplace = True)
df_test.head()

,id,keyword,text
0,0,,Just happened a terrible car crash
1,2,,"Heard about #earthquake is different cities, s..."
2,3,,"there is a forest fire at spot pond, geese are..."
3,9,,Apocalypse lighting. #Spokane #wildfires
4,11,,Typhoon Soudelor kills 28 in China and Taiwan
